# DS Course 9 Wk 3 PA - Segment and Cluster AT

In [22]:
#Libraries for the program
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim

## Read wikipedia data table to a dataframe

In [68]:
dfs=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
CAN_neigh=dfs[0]
CAN_neigh.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Three Key Steps are completed below: 
1. Remove rows with "Not Assiged" boroughs
2. Assign borough name to neighborhood name, if its "Not Assinged"
3. Consolidate rows with same PostCode and Borough 

In [69]:
pd.options.mode.chained_assignment = None  # default='warn'
CAN_neigh = CAN_neigh[CAN_neigh.Borough != "Not assigned"]
CAN_neigh['Neighbourhood']= np.where(CAN_neigh['Neighbourhood'] == "Not assigned",CAN_neigh['Borough'], CAN_neigh['Neighbourhood'])
CAN_neigh['Neighbourhood']= CAN_neigh[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x))
CAN_neigh_final=CAN_neigh[['Postcode','Borough','Neighbourhood']].drop_duplicates()
CAN_neigh_final.rename(columns={'Postcode':'Postalcode'},inplace=True)

In [70]:
CAN_neigh_final.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park


In [72]:
CAN_neigh_final.shape

(103, 3)

# Section 2 - Merging location data

In [73]:
!wget -q -O 'Geocoder_CAN.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [74]:
Geo_df = pd.read_csv('Geocoder_CAN.csv')
Geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
Geo_df.head()


,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
result = pd.merge(CAN_neigh_final, Geo_df, how='inner', on=['Postalcode'])
result.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [76]:
CAN_neigh_final.shape

(103, 3)